In [85]:
#importing libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd

In [86]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [87]:
#parameters
learning_rate = 0.001
batch_size = 16

In [88]:
#Applying Transformations
train_transformations = transforms.Compose([
    transforms.Resize(128),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [89]:
test_transformations = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [90]:
#loading and labelling training set
train_set1 = datasets.ImageFolder("C://Users/Atharva/Dataset/train", transform = train_transformations)
#train_set2 = datasets.ImageFolder("C://Users/Atharva/Dataset/train", transform = train_transformations2)
#train_set = torch.utils.data.ConcatDataset([train_set1])
train_dataloader = torch.utils.data.DataLoader(train_set1, batch_size=batch_size, shuffle=True, num_workers=0)

In [91]:
classes = ['basketball_court', 'beach', 'forest', 'others', 'railway', 'swimming_pool', 'tennis_court']

#used for rearranging predictions as the classes is sorted in alphabetical order and we need according to the classes mentioned in the competition
req_classes = [1,2,3,7,4,6,5]

test_set = datasets.ImageFolder("C://Users/Atharva/Dataset/test_set", transform = test_transformations)

test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=0)

In [92]:
#testing the code
images, labels = next(iter(train_dataloader))

In [93]:
#for i in range(batch_size):
#    test_image = (images[i]*0.5 + 0.5).permute(1, 2, 0)
#    plt.imshow(test_image.numpy())
#    plt.show()

In [94]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.droput_cnn = nn.Dropout(p=0.25)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.batchnorm5 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(in_features=256 * 2 * 2, out_features=1536)
        self.droput_fcc = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=1536, out_features=256)
        self.out = nn.Linear(in_features=256, out_features=7)

    def forward(self, t):
        # Input Layer
        x = t
        
        # 1st Hidden Layer
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # 2nd Hidden Layer
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = self.droput_cnn(x)
        
        # 3rd Hidden layer
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # 4th Hidden layer
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = self.droput_cnn(x)
        
        # 5th Hidden layer
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        # 1st FCl
        x = x.reshape(-1, 256 * 2 * 2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.droput_fcc(x)
        
        # 2nd FCL
        x = self.fc2(x)
        x = F.relu(x)
        x = self.droput_fcc(x)

        output = F.log_softmax(self.out(x),dim = 1)
        
        return output

network = Network().to(device)

In [95]:
#mention the loss function
lambd = 1e-5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(),lr = learning_rate, weight_decay = lambd) # Calculate Loss
#optimizer = optim.SGD(network.parameters(), lr= learning_rate, momentum=0.9, weight_decay = lambd)

In [96]:
def train():
    correct = 0
    total = 0
    running_loss = 0.0
    
    network.train()
    for data in train_dataloader:
        # get the inputs; data is a list of [inputs, labels]
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        
    network.eval()
    with torch.no_grad():
        for data in train_dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = network(images)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return running_loss/len(train_dataloader.sampler), correct*100/total

def test():
    correct = 0
    total = 0
    running_loss = 0
    network.eval()# it-disables-dropout
    with torch.no_grad():
        for data in test_dataloader:
            
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = network(images)
            
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return running_loss/len(test_dataloader.sampler), correct*100/total

In [97]:
no_of_epochs = 170
best_accuracy = 0
for i in range(1, no_of_epochs + 1):
    network.train()
    train_loss, train_acc = train()
    network.eval()
    test_loss, test_acc = test()
    if test_acc > best_accuracy:
        torch.save(network.state_dict(), 'best_model_parameters_1.ckpt')
        best_accuracy = test_acc
        print("Updating Model: ",best_accuracy)
    else:
        print("Not changed: ",best_accuracy)
    print("Epoch: {}    Train_loss.: {:.4f}   Train_acc: {:.2f}   Test_loss.: {:.4f}   Test_acc: {:.2f}".format(i, train_loss,train_acc,test_loss,test_acc))

Updating Model:  18.94736842105263
Epoch: 1    Train_loss.: 0.1128   Train_acc: 20.18   Test_loss.: 0.1292   Test_acc: 18.95
Updating Model:  24.210526315789473
Epoch: 2    Train_loss.: 0.1087   Train_acc: 38.93   Test_loss.: 0.1019   Test_acc: 24.21
Updating Model:  33.68421052631579
Epoch: 3    Train_loss.: 0.1000   Train_acc: 39.64   Test_loss.: 0.0902   Test_acc: 33.68
Not changed:  33.68421052631579
Epoch: 4    Train_loss.: 0.0932   Train_acc: 50.54   Test_loss.: 0.0875   Test_acc: 32.63
Updating Model:  35.78947368421053
Epoch: 5    Train_loss.: 0.0894   Train_acc: 35.18   Test_loss.: 0.0919   Test_acc: 35.79
Updating Model:  38.94736842105263
Epoch: 6    Train_loss.: 0.0897   Train_acc: 50.36   Test_loss.: 0.0858   Test_acc: 38.95
Not changed:  38.94736842105263
Epoch: 7    Train_loss.: 0.0826   Train_acc: 48.57   Test_loss.: 0.0994   Test_acc: 26.32
Not changed:  38.94736842105263
Epoch: 8    Train_loss.: 0.0805   Train_acc: 49.11   Test_loss.: 0.0842   Test_acc: 34.74
Not chan

Not changed:  83.15789473684211
Epoch: 68    Train_loss.: 0.0194   Train_acc: 92.50   Test_loss.: 0.0330   Test_acc: 78.95
Not changed:  83.15789473684211
Epoch: 69    Train_loss.: 0.0167   Train_acc: 95.00   Test_loss.: 0.0396   Test_acc: 81.05
Not changed:  83.15789473684211
Epoch: 70    Train_loss.: 0.0173   Train_acc: 93.39   Test_loss.: 0.0454   Test_acc: 82.11
Not changed:  83.15789473684211
Epoch: 71    Train_loss.: 0.0244   Train_acc: 95.18   Test_loss.: 0.0471   Test_acc: 78.95
Not changed:  83.15789473684211
Epoch: 72    Train_loss.: 0.0186   Train_acc: 95.00   Test_loss.: 0.0476   Test_acc: 77.89
Not changed:  83.15789473684211
Epoch: 73    Train_loss.: 0.0198   Train_acc: 94.29   Test_loss.: 0.0329   Test_acc: 82.11
Not changed:  83.15789473684211
Epoch: 74    Train_loss.: 0.0191   Train_acc: 92.32   Test_loss.: 0.0657   Test_acc: 75.79
Updating Model:  86.3157894736842
Epoch: 75    Train_loss.: 0.0188   Train_acc: 94.11   Test_loss.: 0.0391   Test_acc: 86.32
Not changed:  

Not changed:  89.47368421052632
Epoch: 135    Train_loss.: 0.0109   Train_acc: 98.04   Test_loss.: 0.0611   Test_acc: 86.32
Not changed:  89.47368421052632
Epoch: 136    Train_loss.: 0.0095   Train_acc: 95.18   Test_loss.: 0.0696   Test_acc: 78.95
Not changed:  89.47368421052632
Epoch: 137    Train_loss.: 0.0131   Train_acc: 85.54   Test_loss.: 0.0833   Test_acc: 73.68
Not changed:  89.47368421052632
Epoch: 138    Train_loss.: 0.0089   Train_acc: 97.50   Test_loss.: 0.0531   Test_acc: 86.32
Not changed:  89.47368421052632
Epoch: 139    Train_loss.: 0.0133   Train_acc: 98.39   Test_loss.: 0.0632   Test_acc: 82.11
Not changed:  89.47368421052632
Epoch: 140    Train_loss.: 0.0107   Train_acc: 98.57   Test_loss.: 0.0578   Test_acc: 84.21
Not changed:  89.47368421052632
Epoch: 141    Train_loss.: 0.0068   Train_acc: 98.39   Test_loss.: 0.0819   Test_acc: 84.21
Not changed:  89.47368421052632
Epoch: 142    Train_loss.: 0.0091   Train_acc: 98.21   Test_loss.: 0.0782   Test_acc: 85.26
Not chan

In [ ]:
#torch.save(network.state_dict(), 'network.ckpt')

In [98]:
network.load_state_dict(torch.load('best_model_parameters_1.ckpt'))

<All keys matched successfully>

In [99]:
test = datasets.ImageFolder("C://Users/Atharva/Dataset/test", transform = test_transformations)
verify_dataloader = torch.utils.data.DataLoader(test, batch_size=95, shuffle=False, num_workers=0)

In [100]:
network.eval()# it-disables-dropout
with torch.no_grad():
    correct=0
    total=0
    for data in test_dataloader:
            
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
            
        outputs = network(images)
            
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print("Accuracy",correct*100/total)

Accuracy 90.52631578947368


In [101]:
images, labels = next(iter(verify_dataloader))

In [102]:
output = []
with torch.no_grad():
    correct=0
    total=0
    for data in verify_dataloader:
            
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
            
        outputs = network(images)
            
        _, predicted = torch.max(outputs.data, 1)
    output = predicted

In [103]:
output

tensor([0, 3, 3, 2, 3, 6, 1, 3, 4, 5, 3, 1, 0, 3, 3, 6, 1, 0, 3, 3, 3, 3, 4, 3,
        4, 0, 3, 3, 5, 2, 1, 3, 3, 5, 0, 0, 3, 3, 0, 3, 3, 3, 1, 4, 3, 1, 5, 0,
        2, 5, 5, 4, 2, 3, 0, 2, 6, 3, 3, 6, 3, 4, 2, 1, 3, 3, 3, 5, 1, 2, 3, 4,
        3, 6, 2, 3, 1, 5, 2, 3, 0, 1, 1, 4, 5, 6, 3, 4, 2, 3, 4, 3, 5, 0, 3],
       device='cuda:0')

In [104]:
classes = []
for i in output:
    classes.append(req_classes[i])

In [105]:
#classes

In [106]:
image_id = list(np.array(range(1001,1096)))

In [107]:
df = pd.DataFrame({"ImageID":image_id,"Label":classes})

In [108]:
df.to_csv("180010014_1.csv",index=False,header=True)